In [ ]:
import rdflib
from neo4j import GraphDatabase



In [103]:

from neo4j import GraphDatabase
import numpy as np



uri = "bolt://localhost:7687"
username = "neo4j"
password = "neo4jkngbq"
database = "neo4j"
driver = GraphDatabase.driver(uri, auth=(username, password))



def get_embeddings_nodes_by_type(tx, node_type):
    try: 
        cursor = tx.run("MATCH (n:" +node_type+") WHERE n.embeddings IS NOT NULL RETURN n")
        nodes_with_embeddings = []
        for record in cursor:
            node = record["n"]
            node_properties = node._properties
            
            #the embeddings is a string of floats with comma seperated, need to be converted to list of floats 
            embeddings = node_properties.get('embeddings')
            
            if("Interface" == node_type):
                nodes_with_embeddings.append({"node_id": node_properties.get('id'), "embeddings": embeddings})
            elif("SubNode" == node_type):
                nodes_with_embeddings.append({"node_id": node_properties.get('ID'), "embeddings": embeddings})
                
        return nodes_with_embeddings
    except Exception as e:
        raise Exception(f"Error retrieving nodes with embeddings: {e}")

def create_relation_between_nodes(tx, subNode_id, interface_id, relation="possible_threat"):


    query = f"""
            MATCH (from:SubNode {{ID:$subNode_id}}), (to:Interface {{id:$interface_id}})
            CREATE (from)-[r:{relation}] -> (to)
    """
    tx.run(query, subNode_id=subNode_id, interface_id=interface_id)
    print("Relation successfully created")
        
def convert_embedding_str_to_list(embedding_str):
    emb_split = embedding_str.split(',')
    emb_list = [float(x) for x in emb_split]
    return emb_list
        
def calculate_cosine_similarity(list1,list2):
    return np.dot(list1, list2) / (np.linalg.norm(list1) * np.linalg.norm(list2))



In [58]:
power_grid_nodes = []
mitre_grid_nodes = []

with driver.session() as session:
    power_grid_nodes = session.write_transaction(get_embeddings_nodes_by_type, "Interface")
    mitre_grid_nodes = session.write_transaction(get_embeddings_nodes_by_type, "SubNode")

/var/folders/kv/8ywy_8nd2z716q65sc27_81r0000gn/T/ipykernel_38232/4019469652.py:5: DeprecationWarning: write_transaction has been renamed to execute_write
  power_grid_nodes = session.write_transaction(get_embeddings_nodes_by_type, "Interface")
/var/folders/kv/8ywy_8nd2z716q65sc27_81r0000gn/T/ipykernel_38232/4019469652.py:6: DeprecationWarning: write_transaction has been renamed to execute_write
  mitre_grid_nodes = session.write_transaction(get_embeddings_nodes_by_type, "SubNode")


In [61]:
similarity_for_both_graphs = []

for power_node in power_grid_nodes:
    for attack_node in mitre_grid_nodes:
        node_similarity = {}
        node_similarity['power_node_id'] =power_node.get("node_id")
        node_similarity['attack_node_id'] =attack_node.get("node_id")        
        if(power_node.get("embeddings") and attack_node.get("embeddings")):
            power_node_emb = convert_embedding_str_to_list(power_node.get("embeddings"))
            attack_node_emb = convert_embedding_str_to_list(attack_node.get("embeddings"))
            cosine_similarity = calculate_cosine_similarity(power_node_emb, attack_node_emb)
            node_similarity['similarity'] = cosine_similarity
        similarity_for_both_graphs.append(node_similarity)


In [65]:
similarity_for_both_graphs.sort(key=lambda x: x['similarity'], reverse=True)

In [90]:
similarity_for_both_graphs

[{'power_node_id': 'dtmi:digitaltwins:ngsi_ld:cim:energy:WindPowerPlant;1',
  'attack_node_id': 'G0112',
  'similarity': 0.7860422612746146},
 {'power_node_id': 'dtmi:digitaltwins:ngsi_ld:cim:energy:PowerElectronicsWindUnit;1',
  'attack_node_id': 'G0112',
  'similarity': 0.7772800117013529},
 {'power_node_id': 'dtmi:digitaltwins:ngsi_ld:cim:energy:WindPowerPlant;1',
  'attack_node_id': 'G0090',
  'similarity': 0.7698064968539263},
 {'power_node_id': 'dtmi:digitaltwins:ngsi_ld:cim:energy:WindGenUnitKind;1',
  'attack_node_id': 'G0112',
  'similarity': 0.768967720910361},
 {'power_node_id': 'dtmi:digitaltwins:ngsi_ld:cim:energy:WindGeneratingUnit;1',
  'attack_node_id': 'G0112',
  'similarity': 0.7683030251212641},
 {'power_node_id': 'dtmi:digitaltwins:ngsi_ld:cim:energy:TapSchedule;1',
  'attack_node_id': 'G0096',
  'similarity': 0.7678510167888584},
 {'power_node_id': 'dtmi:digitaltwins:ngsi_ld:cim:energy:TapSchedule;1',
  'attack_node_id': 'G0087',
  'similarity': 0.7653383675117905}

In [104]:
# create relation


for similarity_dict in similarity_for_both_graphs:
    if(similarity_dict.get('similarity') and similarity_dict.get('similarity')>0.75):
        attack_id = similarity_dict.get('attack_node_id')
        power_id = similarity_dict.get('power_node_id')
        with driver.session() as session:
            session.write_transaction(create_relation_between_nodes,attack_id, power_id)

/var/folders/kv/8ywy_8nd2z716q65sc27_81r0000gn/T/ipykernel_38232/1844272257.py:9: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_relation_between_nodes,attack_id, power_id)


Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation successfully created
Relation s

In [19]:
#s = "54321"
str_dict = {}
for i in s:
    if None != str_dict.get(i):
        str_dict[i]+=1
    else:
        str_dict[i] = 1
keys = list(str_dict.keys())
keys.sort(key = lambda x: int(x), reverse=True)
first_half = ''
for j in keys:
    if(str_dict[j]>1 and (j !='0')):
        a = str_dict[j]//2
        first_half += j*a
        str_dict[j] -= 2*a
if(None != str_dict.get('0') and len(first_half)>0):
    a = str_dict['0']//2
    first_half += '0'*a
    str_dict['0'] -= 2*a
second_half = first_half[::-1]
middle=""
for k in keys:
    if str_dict[k]>0:
        middle = k
        break
print(first_half+middle+second_half)

5


In [3]:
print('1'*8)

11111111


In [30]:
from collections import deque
s = '3 DUP 5 - -'
try:
    s_list = s.split(" ")
    num_stack = deque()
    for i in s_list:

        if(i.isdigit()):
            num_stack.append(int(i))
        elif('DUP' == i):
            dup = num_stack.pop()
            num_stack.append(dup)
            num_stack.append(dup)
        elif('POP' == i):
            num_stack.pop()
        elif('+' == i):
            f = num_stack.pop()
            s = num_stack.pop()
            num_stack.append(f+s)
        elif('-' == i):
            f = num_stack.pop()
            s = num_stack.pop()
            num_stack.append(f-s)
    print (num_stack.pop())
except Exception:
    print (-1)
        

-1


In [ ]:
db.users.find({
  $and: [
    { email: { $regex: /@.+?\..+/ } },  
    { "lastLogin.coord.0": { $gte: -90, $lte: 0 } },  
    { "lastLogin.coord.1": { $gte: -180, $lte: 0 } }, 
    { unsuccessfulAttempts: { $gte: 1 } }, 
    { role: { $nin: ["admin", "client"] } },  
    {
      $expr: {
        $and: [
          { $size: "$activeSessions" },  // Check if there are any active sessions
          { $gte: [{ $sum: "$activeSessions.duration" }, 16] } // Sum durations >= 16 (2 sessions * 8 hours)
        ]
      }
    }
  ]
})

In [ ]:
db.users.find({
    "email": { "$regex": /@.+?\..+/},
    "lastLogin.coord.0": { "$gte": -90, "$lte": 0 },
    "lastLogin.coord.1": { "$gte": -180, "$lte": 0 },
    "unsuccessfulAttempts": { "$gte": 1 },
    "role": { "$nin": ["admin", "client"] },
    "activeSessions": { "$size": { "$gte": 2 } },
    {"activeSessions": {
        "$elemMatch": {
            "duration": { "$gte": 8 }
        }
    }}
})

In [ ]:
db.users.find({
    mail: { $regex: /@.+?\..+/ },
    "lastLogin.coord.0": { $gte: -90, $lte: 0 },
    "lastLogin.coord.1": { "$gte": -180, "$lte": 0 },
    "unsuccessfulAttempts": { "$gte": 1 },
    "role": { "$nin": ["admin", "client"] },
    "activeSessions": { $size: { $gte: 2 } },
    activeSessions: {
        $elemMatch: {
            duration: { $gte: 8 }
        }
    }
})

In [ ]:
db.users.find({
    "email": { "$regex": /@.*\./ },
    "lastLogin.coord.0": { "$gte": -90, "$lte": 0 },
    "lastLogin.coord.1": { "$gte": -180, "$lte": 0 },
    "unsuccessfulAttempts": { "$gte": 1 },
    "role": { "$nin": ["admin", "client"] },
    "activeSessions": { "$exists": true, "$not": { "$size": 0 } },
    "$expr": {
        "$gte": [
            { "$size": { "$filter": { "input": "$activeSessions", "as": "session", "cond": { "$gte": ["$$session.duration", 8] } } } },
            2
        ]
    }
})